In [12]:
import datetime
from scipy import optimize

def secant_method(tol, f, x0):
    """
    Solve for x where f(x)=0, given starting x0 and tolerance.

    Arguments
    ----------
    tol: tolerance as percentage of final result. If two subsequent x values are with tol percent, the function will return.
    f: a function of a single variable
    x0: a starting value of x to begin the solver
    Notes
    ------
    The secant method for finding the zero value of a function uses the following formula to find subsequent values of x.

    x(n+1) = x(n) - f(x(n))*(x(n)-x(n-1))/(f(x(n))-f(x(n-1)))

    Warning
    --------
    This implementation is simple and does not handle cases where there is no solution. Users requiring a more robust version should use scipy package optimize.newton.
    """

    x1 = x0 * 1.1
    while (abs(x1 - x0) / abs(x1) > tol):
        x0, x1 = x1, x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0))
    return x1


def xnpv(rate, cashflows):
    """
    Calculate the net present value of a series of cashflows at irregular intervals.
    Arguments
    ---------
    * rate: the discount rate to be applied to the cash flows
    * cashflows: a list object in which each element is a tuple of the form (date, amount), where date is a python datetime.date object and amount is an integer or floating point number. Cash outflows (investments) are represented with negative amounts, and cash inflows (returns) are positive amounts.

    Returns
    -------
    * returns a single value which is the NPV of the given cash flows.
    Notes
    ---------------
    * The Net Present Value is the sum of each of cash flows discounted back to the date of the first cash flow. The discounted value of a given cash flow is A/(1+r)**(t-t0), where A is the amount, r is the discout rate, and (t-t0) is the time in years from the date of the first cash flow in the series (t0) to the date of the cash flow being added to the sum (t).
    * This function is equivalent to the Microsoft Excel function of the same name.
    """

    chron_order = sorted(cashflows, key=lambda x: x[0])
    t0 = chron_order[0][0]  # t0 is the date of the first cash flow

    return sum([cf / (1 + rate) ** ((t - t0).days / 365.0) for (t, cf) in chron_order])


def xirr(cashflows, guess=0.3):
    """
    Calculate the Internal Rate of Return of a series of cashflows at irregular intervals.
    Arguments
    ---------
    * cashflows: a list object in which each element is a tuple of the form (date, amount), where date is a python datetime.date object and amount is an integer or floating point number. Cash outflows (investments) are represented with negative amounts, and cash inflows (returns) are positive amounts.
    * guess (optional, default = 0.1): a guess at the solution to be used as a starting point for the numerical solution.
    Returns
    --------
    * Returns the IRR as a single value

    Notes
    ----------------
    * The Internal Rate of Return (IRR) is the discount rate at which the Net Present Value (NPV) of a series of cash flows is equal to zero. The NPV of the series of cash flows is determined using the xnpv function in this module. The discount rate at which NPV equals zero is found using the secant method of numerical solution.
    * This function is equivalent to the Microsoft Excel function of the same name.
    * For users that do not have the scipy module installed, there is an alternate version (commented out) that uses the secant_method function defined in the module rather than the scipy.optimize module's numerical solver. Both use the same method of calculation so there should be no difference in performance, but the secant_method function does not fail gracefully in cases where there is no solution, so the scipy.optimize.newton version is preferred.
    """

    # return secant_method(0.0001,lambda r: xnpv(r,cashflows),guess)
    return optimize.root(lambda r: xnpv(r, cashflows), guess)




import pandas as pd
import dateutil

data = pd.read_csv(r'C:\Users\eyyup.direk\Desktop\Data_for_EIR_202002071256.csv', header=0)
data['cashflow_date'] = pd.to_datetime(data['cashflow_date'])
# data.dtypes()
#data['CASHFLOW_DATE'] = data['CASHFLOW_DATE'].apply(dateutil.parser.parse, dayfirst=True).dt.date

data['cf_tuple'] = data[['cashflow_date', 'cashflow_amount']].apply(tuple, axis=1)

#print(data.shape)
print(data.dtypes)
#print(data)

#dataout = data.groupby(['contract_id']).apply(xirr(data['cf_tuple']))
#print(dataout)

data_out_from_loop = []
for i, d in data.groupby(['contract_id', 'system_id']):
#    print(type(d.cf_tuple.values))
   #cash_flow = data.cf_tuple.values # data['cf_tuple']
   #print('id:', i, 'irr:', xirr(data.cf_tuple.values))
    try:
       eir = xirr(d.cf_tuple.values)
       modified_eir = (pow(1 + eir.x[0], 1 / 12) - 1 ) * 12
       data_out_from_loop.append( (int(i[0]), int(i[1]), round(eir.x[0],4), round(modified_eir,4) ) )
    except (RuntimeError, TypeError, NameError,AttributeError):
        print(d,eir)
        
#print(data_out_from_loop)

#apply(data_out_from_loop.Series)
#data_out_from_loop[['contract_id','xirr']] = data_out_from_loop[0].apply(data_out_from_loop.series)

data_out_from_loop_final = pd.DataFrame(data_out_from_loop, columns=['contract_id', 'system_id', 'eir', 'modified_eir'])
data_out_from_loop_final.head(4)
print(data_out_from_loop_final.dtypes)
# data_out_from_loop_final = pd.DataFrame(data_out_from_loop, columns=['CONTRACT_ID', 'SYSTEM_ID', 'eir', 'modified_eir'])
# data_out_from_loop_final.info(memory_usage='deep')



OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 8913-05-12 00:00:00

In [ ]:
%debug

> <ipython-input-1-87d83cc91580>(103)<module>()
    101        eir = xirr(d.cf_tuple.values)
    102        modified_eir = (pow(1 + eir.x.item(0), 1 / 12) - 1 ) * 12
--> 103        data_out_from_loop.append( (int(i[0]), int(i[1]), round(eir[0],4), round(modified_eir,4) ) )
    104     except (RuntimeError, TypeError, NameError,AttributeError):
    105         print(d,eir)



ipdb>  eir


    fjac: array([[-1.]])
     fun: array([1.13686838e-13])
 message: 'The solution converged.'
    nfev: 12
     qtf: array([1.5046453e-10])
       r: array([164.56256101])
  status: 1
 success: True
       x: array([-0.27655868])


In [22]:
from io import StringIO, BytesIO


data = ('col1,col2,col3\n' 'a,b,1\n' 'a,b,2\n'  'c,d,3')

pd.read_csv(StringIO(data))

pd.read_csv(StringIO(data), usecols=lambda x: x.upper() in ['COL1', 'COL3'])

,col1,col3
0,a,1
1,a,2
2,c,3


In [12]:
import numpy as np
a = np.array(-1)
type(a)

numpy.ndarray

In [33]:
name = [ "Manjeet", "Nikhil", "Shambhavi", "Astha" ] 
roll_no = [ 4, 1, 3, 2 ] 
marks = [ 40, 50, 60, 70 ]

list(zip(name, roll_no, marks) )

[('Manjeet', 4, 40), ('Nikhil', 1, 50), ('Shambhavi', 3, 60), ('Astha', 2, 70)]

In [41]:
from datetime import date

str(date.today().year) +'{:02d}'.format( date.today().month-1) if date.today().month !=1 else str(date.today().year-1) +'{:02d}'.format(12)

'201912'

In [45]:
df = pd.DataFrame({'A': [1, 2, 3, 4]},
                   index=pd.date_range('2012-08-02', periods=4))
df.resample('2D')

DatetimeIndexResampler [freq=<2 * Days>, axis=0, closed=left, label=left, convention=start, base=0]

In [15]:
df = pd.DataFrame({"A": list("aaabba")})
df.groupby(["A", [1,1,2,3,2,1]]).groups

dtype('O')